In [2]:
# Copyright (c) 2019 Skymind AI Bhd.
# Copyright (c) 2020 CertifAI Sdn. Bhd.
#
# This program and the accompanying materials are made available under the
# terms of the Apache License, Version 2.0 which is available at
# https://www.apache.org/licenses/LICENSE-2.0.
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.
#
# SPDX-License-Identifier: Apache-2.0

# Linear Algebra with NumPy

We'll be using the numerical computing library NumPy, as well as the scientific and technical computing library SciPy. 

In [3]:
import numpy as np 
import scipy.linalg as la

## Operations 
### Matrix transposition 

The transposed matrix A is denoted $A^{T}$.

In [4]:
A = np.array([[1, 2], [3, 4]])
A.T 

array([[1, 3],
       [2, 4]])

### Element-wise product
The element-wise product of A and B is denoted $AB$. It multiplies the values of two matrices element-wise.

In [5]:
B = np.array([[5, 6], [7, 8]])

# in the previous notebook, we used np.matmul(A, B)
# we can also use Python's multiplication operator
A * B

array([[ 5, 12],
       [21, 32]])

### Matrix product 

The matrix product operator is `@`. The inner product of A and B is denoted $AB^{T}$. 

In [6]:
# in the previous notebook, we used np.multiply(A, B) 
# in this notebook we'll use the `@` operator provided by NumPy 
A @ B

array([[19, 22],
       [43, 50]])

### Matrix inversion 

Recall the definition of an inverse matrix: 

$$ AA^{-1} = A^{-1}A = I$$

Use `scipy.linalg` to find the inverse programmatically:

In [7]:
A_inv = la.inv(A)
print(A_inv)

[[-2.   1. ]
 [ 1.5 -0.5]]


In [8]:
A @ A_inv

array([[1.0000000e+00, 0.0000000e+00],
       [8.8817842e-16, 1.0000000e+00]])

which is the identity matrix. 

For a matrix to be invertible, it must be a square matrix. The inverse of a $2\times2$ matrix A is

$$A^{-1}=\begin{bmatrix} a & b \\ c & d \end{bmatrix}^{-1} = \frac{1}{ad - bc} \begin{bmatrix} d & -b \\ -c & a \end{bmatrix}$$

Not all square matrices are invertible. As can be seen from the formula, if $\det{(A)} = ad-bc = 0$, the inverse of A does not exist ($1/0$). 

In [9]:
noninvert = np.array([[6, 4], [3, 2]])
la.det(noninvert)

0.0

### Permutation matrices

Recall that a permutation matrix is just an identity matrix with its rows reordered. A permutation matrix allows us to swap the rows and columns of a matrix. 



When multiplied with an identity matrix, a $n\times n$ square matrix $A$ returns itself: 

$$AI_n=A$$

In [10]:
p = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
p

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [11]:
mat = np.array([[6, 4, 3], [2, 9, 8], [1, 3, 3]])
mat

array([[6, 4, 3],
       [2, 9, 8],
       [1, 3, 3]])

In [12]:
p @ mat

array([[6, 4, 3],
       [2, 9, 8],
       [1, 3, 3]])

Note that (left) multiplying any matrix by a permutation matrix rearranges its rows: 

In [13]:
p1 = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
p1 @ mat

array([[6, 4, 3],
       [1, 3, 3],
       [2, 9, 8]])

and right multiplication rearranges its *columns*: 

In [14]:
mat @ p1

array([[6, 3, 4],
       [2, 8, 9],
       [1, 3, 3]])

## Solving systems of linear equations

Recall in the slides that we tried to solve the following system of linear equations: 

$$x_1+5x_2=7$$
$$-2x_1-7x_2=-5$$

The matrix representation of the above equations are as follows: 

In [15]:
A = np.array([[1, 5], [-2, -7]])
B = np.array([[7, -5]])

We can solve this programmatically with `scipy.linalg`: 

In [16]:
la.solve(A, B.T)

array([[-8.],
       [ 3.]])

Convince yourself that the above values ($x_1 = -8, x_2 = 3$) satisfy the system of linear equations above. 

### Gaussian elimination

In the lectures, we showed how the Gaussian elimination algorithm can be used to solve systems of linear equations. 

Gaussian elimination involves performing the following **elementary row operations**: 
1. Add $k$ times row $j$ to row $i$.
2. Multiply row $i$ by scalar $k$.

The following helper functions are sourced from [patrickwalls/mathematical-python](https://www.math.ubc.ca/~pwalls/math-python/linear-algebra/solving-linear-systems/)

In [17]:
def add_row(A, multiplier_k, row_j, row_i):
    "Add k times row j to row i in matrix A."
    n = A.shape[0]
    E = np.eye(n)
    if row_i == row_j:
        E[row_i, row_i] = multiplier_k + 1
    else:
        E[row_i, row_j] = multiplier_k
    return E @ A


def scale_row(A, row_i, multiplier_k):
    "Multiply row i by k in matrix A."
    n = A.shape[0]
    E = np.eye(n)
    E[row_i, row_i] = multiplier_k
    return E @ A

First, concatenate the two matrices:

In [18]:
E = np.concatenate((A, B.T), axis=1)
E

array([[ 1,  5,  7],
       [-2, -7, -5]])

Remember, the goal is to transform A into an identity matrix. We usually do this by transforming A into an upper triangular matrix (a matrix with 0s below the matrix diagonals) using the elementary operations, *and then* into an identity matrix. 

Our first operation is to add twice of the first row to the second row. Now, the first element in the second row is 0. 

In [19]:
E1 = add_row(E, 2, 0, 1)
E1

array([[1., 5., 7.],
       [0., 3., 9.]])

Now we can scale down row 2 by a factor of 3 ... 

In [20]:
E2 = scale_row(E1, 1, 1/3)
E2

array([[1., 5., 7.],
       [0., 1., 3.]])

and subtract five times of row 2 from the first row. 

Now we have an identity matrix in the first two columns of E3, and the solution in the last column.

In [21]:
E3 = add_row(E2, -5, 1, 0)
E3

array([[ 1.,  0., -8.],
       [ 0.,  1.,  3.]])

which is identical to our programmatic solution. 

### Permutation matrices and Gaussian elimination

Permutation matrices allow us to bypass a limitation of the Gaussian elimination algorithm. 

Take for example the following system of equations 

$$2x_2 = 4$$
$$3x_1-2x_2=5$$

This system of equations clearly has a solution, but let's see how this would be solved under Gaussian elimination. 

In [22]:
A = np.array([[0, 2], [3, -2]])
B = np.array([[4], [5]])

In [25]:
E = np.concatenate((A, B), axis=1)
E

array([[ 0,  2,  4],
       [ 3, -2,  5]])

One way to solve this is to switch the first and second rows. We define a helper function to switch rows: 

In [26]:
def switch_rows(A, row_i, row_j):
    "Switch rows i and j in matrix A."
    n = A.shape[0]
    E = np.eye(n)
    E[row_i, row_i] = 0
    E[row_j, row_j] = 0
    E[row_i, row_j] = 1
    E[row_j, row_i] = 1
    return E @ A

In [27]:
E1 = switch_rows(E, 0, 1)
E1

array([[ 3., -2.,  5.],
       [ 0.,  2.,  4.]])

Now add the two rows together, and scale each of the rows: 

In [28]:
E2 = add_row(E1, 1, 1, 0)
E2

array([[3., 0., 9.],
       [0., 2., 4.]])

In [29]:
E3 = scale_row(E2, 0, 1/3)
E4 = scale_row(E3, 1, 1/2)
E4

array([[1., 0., 3.],
       [0., 1., 2.]])

## Norms

In [30]:
A = np.array([0, 0])
B = np.array([3, 4])

The L1 norm, also known as the Manhattan distance, can be computed using `la.norm`: 

In [31]:
la.norm(B - A, ord=1)

7.0

The Manhattan distance is $\sum_{i=1}^n |x_i-y_i|$.

In contrast, the L2 norm, also known as the Euclidean distance, is $\sqrt{\sum_{i=1}^n (x_i-y_i)^2}$.     

In [32]:
la.norm(B - A, ord=None)

5.0

## Exercise

$$ XA = B$$

Given matrix $A$ and $B$, find matrix $X$. 


In [33]:
A = np.array([
    [2, 5, 6, 1],
    [2, 4, 5, 2],
    [3, 5, 6, 3],
    [6, 4, 2, 7]
])
A

array([[2, 5, 6, 1],
       [2, 4, 5, 2],
       [3, 5, 6, 3],
       [6, 4, 2, 7]])

In [34]:
B = np.array([2, 4, 5, 6])
B

array([2, 4, 5, 6])

In [35]:
B @ la.inv(A)

array([ -2.66666667,  19.66666667, -13.33333333,   1.33333333])

## References
1. https://www.math.utah.edu/~zwick/Classes/Fall2012_2270/Lectures/Lecture7.pdf
2. http://www.math.ubc.ca/~pwalls/math-python/
3. https://github.com/rasbt/pattern_classification/blob/master/resources/latex_equations.md